# Example of the Bert4Rec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [ ]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import Bert4Rec
from replay.models.nn.sequential.bert4rec import (
    Bert4RecPredictionDataset,
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionBatch
)

import pandas as pd

## Getting a spark session

In [ ]:
spark_session = get_spark_session()

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [ ]:
!pip install rs-datasets

In [4]:
from rs_datasets import MovieLens

In [5]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [6]:
interactions.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
user_features.head()

,user_id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [8]:
item_features.head()

,item_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [9]:
interactions["timestamp"] = interactions["timestamp"].astype("int64")
interactions = interactions.sort_values(by="timestamp")
interactions["timestamp"] = interactions.groupby("user_id").cumcount()
interactions

,user_id,item_id,rating,timestamp
1000138,6040,858,4,0
1000153,6040,2384,4,1
999873,6040,593,5,2
1000007,6040,1961,4,3
1000192,6040,2019,5,4
...,...,...,...,...
825793,4958,2399,1,446
825438,4958,1407,5,447
825724,4958,3264,4,448
825731,4958,2634,3,449


### Split interactions into the train, validation and test datasets using LastNSplitter

In [10]:
splitter = LastNSplitter(
    N=1,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

### Prepare FeatureSchema required to create Dataset

In [11]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [12]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [13]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [14]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [15]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        cardinality=train_dataset.item_count,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
        embedding_dim=300,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [ ]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [ ]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [18]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

{'user_id': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 

## Train model
### Create Bert4Rec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 100
BATCH_SIZE = 512
NUM_WORKERS = 4

model = Bert4Rec(
    tensor_schema,
    block_count=2,
    head_count=4,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)
checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

csv_logger = CSVLogger(save_dir=".logs/train", name="Bert4Rec_example")

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=Bert4RecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=Bert4RecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

The path to the best model is saved inside checkpoint_callback

In [20]:
best_model = Bert4Rec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [21]:
prediction_dataloader = DataLoader(
    dataset=Bert4RecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="Bert4Rec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)


trainer = L.Trainer(
    callbacks=[spark_prediction_callback, pandas_prediction_callback, torch_prediction_callback], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(model, dataloaders=prediction_dataloader)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()

In [23]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|    360|12.511995315551758|
|      0|   2020| 11.75175666809082|
|      0|   2012|11.454107284545898|
|      0|   2027|11.378838539123535|
|      0|   1642| 11.18391227722168|
|      0|    592|10.691895484924316|
|      0|   1012|10.673810958862305|
|      0|   1019| 10.30530834197998|
|      0|   1382|10.099740982055664|
|      0|   2325|10.061320304870605|
|      0|   2018|  9.99974250793457|
|      0|   1850| 9.938338279724121|
|      0|   2016| 9.866159439086914|
|      0|   2014| 9.860157012939453|
|      0|   2011| 9.853334426879883|
|      0|   2009| 9.806418418884277|
|      0|   2070| 9.749092102050781|
|      0|     12| 9.694207191467285|
|      0|    105| 9.655630111694336|
|      0|   2021| 9.566499710083008|
+-------+-------+------------------+
only showing top 20 rows



In [24]:
pandas_res

,user_id,item_id,score
0,0,360,12.511995
0,0,2020,11.751757
0,0,2012,11.454107
0,0,2027,11.378839
0,0,1642,11.183912
...,...,...,...
6039,6039,377,5.304715
6039,6039,1816,5.291126
6039,6039,1170,5.27256
6039,6039,1593,5.270464


In [25]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([ 360, 2020, 2012, 2027, 1642,  592, 1012, 1019, 1382, 2325, 2018, 1850,
        2016, 2014, 2011, 2009, 2070,   12,  105, 2021,  547,   47,  310, 2285,
        3090, 3546, 1262, 2631,  826, 3144, 2068, 1020, 2965, 2023,  700, 1990,
        1664, 2072,  582, 2692, 1050, 1011,  259,  612, 1010,  974, 3414, 1393,
        2788,   33, 1346,  731, 2315,   59, 2830, 1547, 1000, 1002, 1964, 3088,
        1979, 2033,  999,  626, 3329, 3682, 1017, 1254,  791, 2030, 2640, 2985,
        1205,  156, 2807, 2073, 3218, 1018,  314, 3328, 2092, 1459, 2054,  358,
        2013,   72, 1043, 1433, 1983,  605, 1058,    1, 3331,  876, 1508, 2931,
        2918, 2069, 1551, 2497]) tensor([12.5120, 11.7518, 11.4541, 11.3788, 11.1839, 10.6919, 10.6738, 10.3053,
        10.0997, 10.0613,  9.9997,  9.9383,  9.8662,  9.8602,  9.8533,  9.8064,
         9.7491,  9.6942,  9.6556,  9.5665,  9.4397,  9.4348,  9.3644,  9.2273,
         9.1622,  9.1302,  9.0864,  9.0727,  9.0493,  9.0057,  8.9362,  8.929

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [26]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [27]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
| 9.938338279724121|      1|   1919|
|10.099740982055664|      1|   1405|
| 9.806418418884277|      1|   2078|
|10.691895484924316|      1|    596|
| 9.566499710083008|      1|   2090|
|  9.99974250793457|      1|   2087|
|10.061320304870605|      1|   2394|
|11.378838539123535|      1|   2096|
|11.454107284545898|      1|   2081|
|10.673810958862305|      1|   1025|
| 9.655630111694336|      1|    107|
| 9.749092102050781|      1|   2139|
| 9.866159439086914|      1|   2085|
| 9.853334426879883|      1|   2080|
| 9.694207191467285|      1|     13|
| 9.860157012939453|      1|   2083|
| 10.30530834197998|      1|   1032|
| 9.439712524414062|      1|    551|
|12.511995315551758|      1|    364|
| 11.18391227722168|      1|   1688|
+------------------+-------+-------+
only showing top 20 rows



### Calculating metrics

In [28]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [29]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [30]:
metrics_to_df(result_metrics)

k,1,10,20,100
HitRate,0.037914,0.196689,0.294868,0.576987
MAP,0.037914,0.076709,0.083444,0.090536
MRR,0.037914,0.076709,0.083444,0.090536
NDCG,0.037914,0.104596,0.129325,0.181368
Precision,0.037914,0.019669,0.014743,0.005770
Recall,0.037914,0.196689,0.294868,0.576987


## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask and tokens mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [31]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
tokens_mask = padding_mask.roll(-1, dims=0)
tokens_mask[-1, ...] = 0
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the Bert4RecPredictionBatch entity

In [32]:
batch = Bert4RecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask,
    features={ITEM_FEATURE_NAME: item_sequence.long()},
    tokens_mask=tokens_mask
)

### Run predict step of the Bert4Rec and get scores from the model

In [33]:
with torch.no_grad():
    scores = model.predict_step(batch, 0)
scores

tensor([[ 1.4643, -1.8794, -1.0493,  ..., -1.6864, -2.8441,  0.3529]])

### Getting five items with the highest score

In [34]:
torch.topk(scores, k=5).indices

tensor([[3684, 2693, 3724, 3107, 3078]])